In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import pandas as pd

Manually remove illegal data.

In [ ]:
ds_split = {
    11: [(2000, 11000), (12000, 21000)],
    12: [(2000, 5000 ), (6000 , 10000)],
    13: [(2000, 10000), (11000, 18000)],
    21: [(2000, 11000), (12000, 20000)],
    22: [(2000, 6000 ), (7000 , 11000)],
    23: [(2000, 11000), (12000, 20000)],
    31: [(2000, 11000), (12000, 20000)],
    32: [(2000, 5000 ), (6000 , 11000)],
    33: [(2000, 10000), (11000, 19000)],
    41: [(2000, 10000), (11000, 19000)],
    42: [(2000, 6000 ), (7000 , 10000)],
    43: [(2000, 10000), (11000, 19000)],
    51: [(2000, 11000), (12000, 20000)],
    52: [(2000, 6000 ), (7000 , 10000)],
    53: [(2000, 10000), (11000, 19000)],
    61: [(2000, 9000 ), (10000, 17000)],
    62: [(2000, 11000), (12000, 20000)],
    63: [(2000, 10000), (11000, 19000)],
    71: [(2000, 7000 ), (9000 , 19000)],
    72: [(2000, 9000 ), (10000, 17000)],
    73: [(2000, 10000), (11000, 19000)]
}

## Generate Data

In [ ]:
COLUMNS = ['timestamp', 'acceleration_x', 'acceleration_y', 'acceleration_z', 
           'input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll',
           'input_orientation_x', 'input_orientation_y', 'input_orientation_z', 'input_orientation_w',
           'magnetic_x', 'magnetic_y', 'magnetic_z',
           'angular_vec_x', 'angular_vec_y', 'angular_vec_z']

In [ ]:
!mkdir data/1220

In [ ]:
all_len, all_yaw, all_pitch, all_roll, all_x, all_y, all_z = 0, 0, 0, 0, 0, 0, 0

for key in ds_split.keys():
    user = key // 10
    scene = key % 10
    csv_fname = f'data/org/motion_data_user{user}_scene{scene}.csv'
    csv = pd.read_csv(csv_fname)

    csv = csv[COLUMNS]

    # timestamp flick ==> sec
    csv.timestamp = (csv.timestamp - csv.timestamp[2000]) / 705600000

    # yaw, pitch, roll radian ==> degree
    csv.input_orientation_yaw *= 180 / pi
    csv.input_orientation_pitch *= 180 / pi
    csv.input_orientation_roll *= 180 / pi
    csv.angular_vec_x *= 180 / pi
    csv.angular_vec_y *= 180 / pi
    csv.angular_vec_z *= 180 / pi
    csv.input_orientation_x *= 180 / pi
    csv.input_orientation_y *= 180 / pi
    csv.input_orientation_z *= 180 / pi
    csv.input_orientation_w *= 180 / pi

    T = csv.timestamp
    dT = T[:-1].to_numpy() - T[1:].to_numpy()
    dT = np.insert(dT, 0, dT[0])

    for i, (S, T) in enumerate(ds_split[key]):
        dsv_fname = f'data/1220/user{user}_scene{scene}_{i}.csv'
        print(dsv_fname)
        dsv = csv.iloc[S:T]
        dsv.to_csv(dsv_fname, index=False)

# Generate *.npz data file

In [ ]:
from pathlib import Path
from torch.utils.data import Dataset
import numpy as np
import pandas as pd

In [ ]:
XCOLS = ['input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll',
         'acceleration_x', 'acceleration_y', 'acceleration_z']
YCOLS = ['input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll']

In [ ]:
train_files = sorted(list(Path('data/1220').glob('*scene3_0.csv')))
test_files = sorted(list(Path('data/1220').glob('*scene3_1.csv')))

In [ ]:
window_size = 120
stride = 1

## Make Train Dataset

In [ ]:
X = []
Y = []
for f in train_files:
    print(f)
    csv = pd.read_csv(f)
    i = 0
    while i <= len(csv) - window_size * 2:
        X.append(csv.iloc[i:i + window_size][XCOLS].to_numpy())
        Y.append(csv.iloc[i + window_size + 18][YCOLS].to_numpy())
        i += stride

In [ ]:
np.savez_compressed(f'data/1220/train-win_{window_size}.npz', X=X, Y=Y)

## Make Test Dataset

In [ ]:
X = []
Y = []
for f in test_files:
    print(f)
    csv = pd.read_csv(f)
    i = 0
    while i <= len(csv) - window_size * 2:
        X.append(csv.iloc[i:i + window_size][XCOLS].to_numpy())
        Y.append(csv.iloc[i + window_size + 18][YCOLS].to_numpy())
        i += stride

In [ ]:
np.savez_compressed(f'data/1220/test-win_{window_size}.npz', X=X, Y=Y)

# Generate *.npz data file for GAN

GAN uses input shape `(BatchSize, Channels(=6), WindowSize(=120))`, and output shape `(BatchSize, Channels(=3), WindowSize(=120))`.

# Make Train Dataset

In [ ]:
X_train = []
Y_train = []
for f in train_files:
    print(f)
    csv = pd.read_csv(f)
    i = 0
    while i <= len(csv) - window_size * 2:
        X_train.append(csv.iloc[i:i + window_size][XCOLS].to_numpy())
        Y_train.append(csv.iloc[i + 18:i + window_size + 18][YCOLS].to_numpy())
        i += stride

In [ ]:
np.savez_compressed(f'data/1220/train-win_{window_size}-GAN.npz', X=X_train, Y=Y_train)

# Make Test Dataset

In [ ]:
X_train = []
Y_train = []
for f in test_files:
    print(f)
    csv = pd.read_csv(f)
    i = 0
    while i <= len(csv) - window_size * 2:
        X_train.append(csv.iloc[i:i + window_size][XCOLS].to_numpy())
        Y_train.append(csv.iloc[i + 18:i + window_size + 18][YCOLS].to_numpy())
        i += 18

In [ ]:
np.savez_compressed(f'data/1220/test-win_{window_size}-GAN.npz', X=X_train, Y=Y_train)